# 新手教程—财务数据

### 我们以查询数据库的方式向您提供了财务数据接口, 调用 get_fundamentals, 传入您要筛选的条件即可. 我们提供了四类财务数据, 分别存放到如下四个表中: 市值表, 利润表, 现金表, 负债表

In [3]:
#导入需要的程序包
import pandas as pd
import seaborn as sns

from jqdatasdk import *

# auth('ID','Password')

auth('13811550483','security421S')
pd.set_option('display.max_rows', 10)

In [4]:
# 使用?快速获取帮助

# 获取财务数据接口帮助
get_fundamentals?

# 注释掉下面的行来查询各个API的介绍
# query?

# # 各个表的介绍
# valuation?
# income?
# balance?
# cash_flow?

# # 各个表的字段的介绍示例
# valuation.pe_ratio?
# income.total_operating_revenue?

In [5]:
### 获取单只股票在某一日期的市值数据
df = get_fundamentals(query(
        valuation
    ).filter(
        valuation.code == '000001.XSHE'
    ), date='2019-10-15')

df

,id,code,pe_ratio,turnover_ratio,pb_ratio,ps_ratio,pcf_ratio,capitalization,market_cap,circulating_cap,circulating_market_cap,day,pe_ratio_lyr
0,39540718,000001.XSHE,12.4174,0.4545,1.4313,2.6189,-5.6196,1940591.875,3333.9368,1940575.25,3333.9082,2019-10-15,13.4335


In [6]:
# 取出总市值
df['market_cap'][0]

3333.9368

In [7]:
### 获取多只股票在某一日期的市值, 利润
df = get_fundamentals(query(
        valuation, income
    ).filter(
        # 这里不能使用 in 操作, 要使用in_()函数
        valuation.code.in_(['000001.XSHE', '600000.XSHG'])
    ), date='2019-10-15')

df

,id,code,pe_ratio,turnover_ratio,pb_ratio,ps_ratio,pcf_ratio,capitalization,market_cap,circulating_cap,...,income_tax_expense,net_profit,np_parent_company_owners,minority_profit,basic_eps,diluted_eps,other_composite_income,total_composite_income,ci_parent_company_owners,ci_minority_owners
0,39540718,000001.XSHE,12.4174,0.4545,1.4313,2.6189,-5.6196,1940591.875,3333.9368,1940575.25,...,2.375000e+09,7.957000e+09,7.957000e+09,NaN,0.47,0.42,1.980000e+08,8.155000e+09,NaN,NaN
1,39542892,600000.XSHG,6.4628,0.2612,0.8301,2.0533,-7.3516,2935208.000,3842.1873,2810376.50,...,3.115000e+09,1.578600e+10,1.564700e+10,139000000.0,0.54,0.54,-1.205000e+09,1.458100e+10,1.444000e+10,141000000.0


In [8]:
# 只选择表里的某些字段
### 获取多只股票在某一日期的市值, 利润, 现金流和负债数据
df = get_fundamentals(query(
        valuation.code, valuation.market_cap, valuation.pe_ratio, income.total_operating_revenue
    ).filter(
        # 这里不能使用 in 操作, 要使用in_()函数
        valuation.code.in_(['000001.XSHE', '600000.XSHG'])
    ), date='2019-10-15')

df

,code,market_cap,pe_ratio,total_operating_revenue
0,000001.XSHE,3333.9368,12.4174,3.535300e+10
1,600000.XSHG,3842.1873,6.4628,4.751500e+10


In [16]:
# 选出所有的总市值大于1000亿元, 市盈率小于10, 营业总收入大于200亿元的股票
df = get_fundamentals(query(
        valuation.code, valuation.market_cap, valuation.pe_ratio, income.total_operating_revenue
    ).filter(
        valuation.market_cap > 1000,
        valuation.pe_ratio < 10,
        income.total_operating_revenue > 2e10
    ), date='2019-10-15')

df

,code,market_cap,pe_ratio,total_operating_revenue
0,000001.XSHE,1598.2800,7.50,2.590400e+10
1,000002.XSHE,1493.9100,9.47,4.137246e+10
2,000333.XSHE,1173.9200,9.61,4.030065e+10
3,000651.XSHE,1049.7500,7.41,2.651671e+10
4,600000.XSHG,2965.8999,6.14,3.771000e+10
5,600016.XSHG,3137.7400,6.86,4.087200e+10
6,600030.XSHG,1904.7800,9.65,2.117836e+10
7,600036.XSHG,4625.3198,7.92,5.338800e+10
8,600104.XSHG,1959.2400,6.86,1.537495e+11
9,600606.XSHG,1844.6899,-243.86,8.609665e+10


In [14]:
# 在回测环境中可用: 选取上面的结果作为universe
# set_universe(list(df['code']))